In [82]:
import cv2
import glob,os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [83]:
# reading images from the data folder
# label, smile = 1, neutral = 0
(img_width, img_height) = (45, 25)
def load_images_from_folder(folder):
    (images, lables, names, id) = ([], [], {}, 0)
    for (subdirs, dirs, files) in os.walk(folder):
        print subdirs, dirs
        for subdir in dirs:
            names[id] = subdir
            subjectpath = os.path.join(folder, subdir)
            for filename in os.listdir(subjectpath):
                path = subjectpath + '/' + filename
                lable = id
                img = cv2.imread(path, 0)
                img = cv2.resize(img,(img_width, img_height))
                images.append(img)
                lables.append(int(lable))
            id += 1
        
        return images, lables, names


In [84]:
X,Y,classes = load_images_from_folder("./data/")

(X,Y) = [np.array(lis) for lis in [X, Y]]
Y = pd.get_dummies(Y) #concerting labels to one-hot, Used Pandas for it. need to do it with numpy
# print Y
# print len(classes)


./data/ ['neutral', 'smile']


In [85]:
print "Height of the image:  " + str(X.shape[1])
print "Width of the image:   "  + str(X.shape[2])
print "Total number of data: "+ str(len(X))


Height of the image:  25
Width of the image:   45
Total number of data: 144


In [86]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

In [87]:
print "lenght of train data: "+str(len(X_train))
print "lenght of test data:  "+str(len(X_test))

lenght of train data: 96
lenght of test data:  48


In [88]:
train_set_x_flatten = X_train.reshape(X_train.shape[0],-1)
test_set_x_flatten = X_test.reshape(X_test.shape[0],-1)
print "train_set_x_flatten: "+str(train_set_x_flatten.shape)
print "test_set_x_flatten : "+str(test_set_x_flatten.shape)

print train_set_x_flatten

index = 35
# plt.imshow(X_train[index])
# print ("y = " + str(y_train[index]) + ", it's a '" + classes[np.squeeze(y_train[index])].decode("utf-8") +  "' picture.")

train_set_x_flatten: (96, 1125)
test_set_x_flatten : (48, 1125)
[[170 181 179 ..., 154 159 155]
 [201 198 199 ..., 181 178 175]
 [143 143 146 ..., 224 225 226]
 ..., 
 [163 164 162 ..., 151 150 150]
 [146 147 143 ...,  14  11   7]
 [184 191 188 ...,  83  70  88]]


In [89]:
#Normalizing data, conveting all pixel value in range between 0-1

train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.
print "length of train set: " + str(len(train_set_x))
print "length of test set: " + str(len(test_set_x))

length of train set: 96
length of test set: 48


In [90]:
print "shape of train set: "+ str(train_set_x.shape)
print "shape of test set: "+ str(test_set_x.shape)

print len(train_set_x), len(y_train)

shape of train set: (96, 1125)
shape of test set: (48, 1125)
96 96


# Building the softmax regression using tensorflow

In [91]:
import tensorflow as tf

In [92]:
x = tf.placeholder(tf.float32, [None, 1125])
W = tf.Variable(tf.zeros([1125, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


In [93]:
print "training with learning rate 0.1 "
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for step in range(15000):
    trainStep,loss = sess.run([train_step, cross_entropy], feed_dict={x: train_set_x, y_: y_train})
    if step%1000==0:
        print "losses after per 1000 iteration: ",loss
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy with learning rate 0.1 ", sess.run(accuracy, feed_dict={x:test_set_x , y_: y_test}))

training with learning rate 0.1 
losses after per 1000 iteration:  0.693147
losses after per 1000 iteration:  0.018768
losses after per 1000 iteration:  0.0110425
losses after per 1000 iteration:  0.00812822
losses after per 1000 iteration:  0.00650153
losses after per 1000 iteration:  0.00544514
losses after per 1000 iteration:  0.00469743
losses after per 1000 iteration:  0.00413739
losses after per 1000 iteration:  0.00370071
losses after per 1000 iteration:  0.00334993
losses after per 1000 iteration:  0.00306153
losses after per 1000 iteration:  0.00281997
losses after per 1000 iteration:  0.00261454
losses after per 1000 iteration:  0.0024376
losses after per 1000 iteration:  0.00228353
('accuracy with learning rate 0.1 ', 0.95833331)


In [94]:
0.555556
0.01

0.01

In [95]:
print "Training with learning rate 0.5"
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for step in range(1000):
    trainStep,loss = sess.run([train_step, cross_entropy], feed_dict={x: train_set_x, y_: y_train})
    if step%200==0:
        print "losses after per 1000 iteration: ",loss
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy with learning rate 0.5 ", sess.run(accuracy, feed_dict={x:test_set_x , y_: y_test}))

Training with learning rate 0.5
losses after per 1000 iteration:  0.693147
losses after per 1000 iteration:  nan
losses after per 1000 iteration:  nan
losses after per 1000 iteration:  nan
losses after per 1000 iteration:  nan
('accuracy with learning rate 0.5 ', 0.45833334)
